In [1]:
# !git clone https://github.com/facebookresearch/fastText.git
# %cd fastText
# !/home/danpr/.conda/envs/nlp2/bin/python -m pip install .
# !git clone https://github.com/NasLabBgu/parler-hate-speech
# !/home/danpr/.conda/envs/nlp2/bin/python -m pip install -U easynmt
# !/home/danpr/.conda/envs/nlp2/bin/python -m pip install thai-segmenter
# %cd ..

In [2]:
from tqdm.auto import tqdm
from easynmt import EasyNMT
import pandas as pd
import torch
model = EasyNMT('m2m_100_418M')

/home/danpr/.conda/envs/nlp2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# def BackTranslation(df,lg='de'):
#     sentences_en = df.text.to_list()
#     translations_lg = model.translate(sentences_en, target_lang=lg)
#     translations_back_translated = model.translate(translations_lg, target_lang='en')
#     return translations_back_translated

# df = pd.read_csv("parler-hate-speech/parler_annotated_data.csv").head(10)

# for lg in tqdm(['de','fr','ko', 'he', 'es']):
#     df[f'BackTranslation_{lg}'] = BackTranslation(df,lg)

In [4]:
# df.to_csv("data_bk.csv",index=False)

In [5]:
# row = df.iloc[3]
# row['text']
# row[f'BackTranslation_de']
# row[f'BackTranslation_ko']
# row[f'BackTranslation_fr']

In [ ]:
"""
This example shows how EasyNMT to stream translations.
Streaming translations can be useful when you want to translate a large set of sentences / documents.
The method chunks the data, translates it, and returns the results.
This can be useful if you want to write it e.g. to a file.
"""

#First, we create a large set of sentences:
df = pd.read_csv("parler-hate-speech/parler_annotated_data.csv")
sentences_english = df.text.to_list()

#The method model.translate_stream chunks the data into sets of size chunk_size
#It then translate these documents/sentences and yields the result
for target_language in ['de', 'fr', 'ko', 'he', 'es']:
    print(target_language)
    sentences_foreign = []
    try:
        for translated_2_foreign in model.translate_stream(sentences_english, show_progress_bar=True, chunk_size=64, target_lang=target_language):
            sentences_foreign.append(translated_2_foreign)
    except Exception as e:
        print(e)

    sentences_backtranslated = []
    try:
        for backtranslated_2_english in model.translate_stream(sentences_foreign, show_progress_bar=True, chunk_size=64, target_lang='en'):
            sentences_backtranslated.append(backtranslated_2_english)
    except Exception as e:
        print(e)

    dest_csv = f'parler-hate-speech/parler_annotated_data_{target_language}.csv'
    pd.DataFrame(
        list(zip(sentences_english, sentences_backtranslated)), 
        columns=['text', f'text_en_{target_language}_en']
    ).merge(df, on='text').to_csv(dest_csv, index=False)

de


 14%|█▍        | 1407/10121 [05:32<34:17,  4.23it/s]


'eo'


100%|██████████| 1344/1344 [04:36<00:00,  4.86it/s]


fr


...

 14%|█▍        | 1407/10121 [05:24<33:29,  4.34it/s]


'eo'


100%|██████████| 1344/1344 [04:05<00:00,  5.47it/s]


es


  6%|▌         | 576/10121 [02:15<37:19,  4.26it/s]